<a href="https://colab.research.google.com/github/HeptaDecane/LP1_SEM7/blob/main/HPC/A04/binary_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [19]:
code = """
#include<iostream>

#include<omp.h>

#include<time.h>

using namespace std;
int c1[3];
int binary_search(int a[], int low, int high, int key) {
    int loc = -1;
    int mid;
    while (low <= high) {
        mid = (high + low) / 2;
        if (a[mid] == key) {
            loc = mid;
            break;
        } 
        else {
            #pragma omp parallel sections 
            {
                //cout<<omp_get_thread_num();
                #pragma omp section 
                {
                    if (a[mid] < key) {
                        low = mid + 1;
                    }
                }
                #pragma omp section 
                {
                    if (a[mid] > key) {
                        high = mid - 1;
                    }
                }
            }
        }
    }
    return loc;
}

int main() {
    int th = omp_get_max_threads();
    cout << "Max Threads : " << th << endl;
    omp_set_num_threads(2);
    c1[1] = 0;
    c1[2] = 0;
    int a[1000000];
    clock_t t1, t2;
    int key = 0;
    int loc, i;
    for (int i = 0; i < 1000000; i++) {
        a[i] = i;
    }
    key = 1000;
    t1 = clock();
    loc = binary_search(a, 0, 1000000, key);
    t2 = clock();
    if (loc == -1) {
        cout << "Not Found";
    } else {
        cout << "Found at " << loc << endl;
        //cout<<"By Thread "<<omp_get_thread_num()<<endl;
    }
    return 0;
}
"""

In [20]:
file = open('binary_search.cpp','w')
file.write(code)
file.close()

In [21]:
!g++ -fopenmp binary_search.cpp

In [22]:
!./a.out

Max Threads : 2
Found at 1000
